In [1]:
import pandas as pd

# Master Country

In [2]:
df=pd.read_csv('all_country.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      249 non-null    object 
 1   alpha-2                   248 non-null    object 
 2   alpha-3                   249 non-null    object 
 3   country-code              249 non-null    int64  
 4   iso_3166-2                249 non-null    object 
 5   region                    247 non-null    object 
 6   sub-region                247 non-null    object 
 7   intermediate-region       105 non-null    object 
 8   region-code               247 non-null    float64
 9   sub-region-code           247 non-null    float64
 10  intermediate-region-code  105 non-null    float64
dtypes: float64(3), int64(1), object(7)
memory usage: 21.5+ KB


,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [3]:
name_to_alpha2 = df.set_index('name')['alpha-2'].to_dict()
str(name_to_alpha2)[:100]

"{'Afghanistan': 'AF', 'Åland Islands': 'AX', 'Albania': 'AL', 'Algeria': 'DZ', 'American Samoa': 'AS"

In [4]:
def mapping1(name):
    import sqlite3
    alpha2=name_to_alpha2[name]
    return alpha2

mapping1('Albania')

'AL'

In [5]:
# Dirty Input

In [6]:
import sqlite3
con = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = con.cursor()
df2=pd.DataFrame(cur.execute('SELECT * FROM Invoice').fetchall())
df2.columns=[i[0] for i in cur.execute('SELECT * FROM Invoice').description]
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   InvoiceId          412 non-null    int64  
 1   CustomerId         412 non-null    int64  
 2   InvoiceDate        412 non-null    object 
 3   BillingAddress     412 non-null    object 
 4   BillingCity        412 non-null    object 
 5   BillingState       210 non-null    object 
 6   BillingCountry     412 non-null    object 
 7   BillingPostalCode  384 non-null    object 
 8   Total              412 non-null    float64
dtypes: float64(1), int64(2), object(6)
memory usage: 29.1+ KB


,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [7]:
df3=df2[['BillingCountry']]
df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   BillingCountry  412 non-null    object
dtypes: object(1)
memory usage: 3.3+ KB


,BillingCountry
0,Germany
1,Norway
2,Belgium
3,Canada
4,USA


# Squishy Data Transformation

In [8]:
# pip install monadsquishy

In [9]:
def mapping2(name):
    another_name_to_alpha2={'USA':'US','United Kingdom':'UK'}
    # another_name_to_alpha2={'USA':'US','United Kingdom':'UK','Netherlands':'NL', 'Czech Republic':'CZ'}
    alpha2=another_name_to_alpha2[name]
    return alpha2


mapping2('USA')

'US'

In [10]:
sq_config = {
    'transformations':[
        {
            'input_table': df3,
            'transformed_path':'./staging/test1',
            'exploded_path':'./staging/test1',
            'out_columns': {
                 'country_out': {
                     'input':'BillingCountry',
                     'funcs':[mapping1, mapping2]
                 } 
             }
        }
    ]
}

In [11]:
import monadsquishy as ms
sq = ms.Squishy(sq_config)
sq.run()

1/1 Output: country_out
Input: BillingCountry       -->['mapping1', 'mapping2']                                                                                                                                                                                

Finished transformations!


In [12]:
sq.clean_report()

,input_column,out_column,message,clean_count
0,BillingCountry,country_out,Passed: mapping1(),279
1,BillingCountry,country_out,Passed: mapping2(),112


In [13]:
sq.dirty_report()

,input_column,out_column,input_value,dirty_count
0,BillingCountry,country_out,Czech Republic,14
1,BillingCountry,country_out,Netherlands,7


In [14]:
sq.log()

,input_row,input_column,output_column,input_value,output_value,is_passed,message
0,0,BillingCountry,country_out,Germany,DE,True,Passed: mapping1()
1,1,BillingCountry,country_out,Norway,NO,True,Passed: mapping1()
2,2,BillingCountry,country_out,Belgium,BE,True,Passed: mapping1()
3,3,BillingCountry,country_out,Canada,CA,True,Passed: mapping1()
4,4,BillingCountry,country_out,USA,US,False,Failed: mapping1(): 'USA'
...,...,...,...,...,...,...,...
540,407,BillingCountry,country_out,USA,US,True,Passed: mapping2()
541,408,BillingCountry,country_out,Canada,CA,True,Passed: mapping1()
542,409,BillingCountry,country_out,Portugal,PT,True,Passed: mapping1()
543,410,BillingCountry,country_out,Finland,FI,True,Passed: mapping1()


In [15]:
df_log = sq.log()
df_log.head(10)

,input_row,input_column,output_column,input_value,output_value,is_passed,message
0,0,BillingCountry,country_out,Germany,DE,True,Passed: mapping1()
1,1,BillingCountry,country_out,Norway,NO,True,Passed: mapping1()
2,2,BillingCountry,country_out,Belgium,BE,True,Passed: mapping1()
3,3,BillingCountry,country_out,Canada,CA,True,Passed: mapping1()
4,4,BillingCountry,country_out,USA,US,False,Failed: mapping1(): 'USA'
5,4,BillingCountry,country_out,USA,US,True,Passed: mapping2()
6,5,BillingCountry,country_out,Germany,DE,True,Passed: mapping1()
7,6,BillingCountry,country_out,Germany,DE,True,Passed: mapping1()
8,7,BillingCountry,country_out,France,FR,True,Passed: mapping1()
9,8,BillingCountry,country_out,France,FR,True,Passed: mapping1()
